<a href="https://colab.research.google.com/github/noritaketakamichi/manabie_auto_allocation_test/blob/add_desired_teacher/auto_allocation_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ==========================================
# 1. ライブラリ導入とGoogle認証
# ==========================================
!pip install ortools gspread pandas --quiet

import gspread
import pandas as pd
from google.colab import auth
from google.auth import default
from ortools.linear_solver import pywraplp
import collections

# 認証処理
print("認証を開始します...")
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# ▼▼▼ スプレッドシートのURL ▼▼▼
SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/1NxNVevKlGBhTfUrVEhMcaUgy4WJWPBiObetuWYX_u70'
# ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲

print("✅ 準備完了。次のセルを実行してデータを読み込んでください。")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 23.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.31.1 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.
認証を開始します...
✅ 準備完了。次のセルを実行してデータを読み込んでください。


In [8]:
# ==========================================
# 2. データの読み込みと「診断レポート」表示
# ==========================================

# シート名の定義
sheet_names = {
    'slots':         'lesson_slot',
    'courses':       'course',
    'students':      'student_list',
    'student_reqs':  'student_course',
    'student_avail': 'student_availability',
    'teachers':      'teacher_list',
    'teachable':     'teachable_courses',
    'teacher_avail': 'teacher_availability'
}

dfs = {}
wb = gc.open_by_url(SPREADSHEET_URL)

print("--- 📥 データを読み込んでいます... ---")
try:
    for key, sheet_name in sheet_names.items():
        try:
            ws = wb.worksheet(sheet_name)
            data = ws.get_all_records()
            dfs[key] = pd.DataFrame(data)
            print(f"・{sheet_name}: {len(dfs[key])}行 読み込みOK")
        except gspread.WorksheetNotFound:
            print(f"⚠️ 警告: シート '{sheet_name}' が見つかりません！")
            dfs[key] = pd.DataFrame() # 空のDFを作成

    # 変数展開
    df_slots = dfs['slots']
    df_students = dfs['students']
    df_teachers = dfs['teachers']
    df_courses = dfs['courses']
    df_reqs = dfs['student_reqs']
    df_s_avail = dfs['student_avail']
    df_t_avail = dfs['teacher_avail']
    df_teachable = dfs['teachable']

    # マッピング作成
    s_map = dict(zip(df_students['id'], df_students['student_name']))
    t_map = dict(zip(df_teachers['id'], df_teachers['teacher_name']))
    c_map = dict(zip(df_courses['id'], df_courses['course_name']))

    print("\n" + "="*40)
    print("📊 データ診断レポート")
    print("="*40)

    # --- 1. 生徒の授業希望チェック ---
    print(f"\n📌 【授業リクエスト】(全 {len(df_reqs)} 件)")
    if not df_reqs.empty:
        # 生徒ごとの希望数集計
        req_summary = df_reqs.groupby('student_id')['sessions'].sum()
        print("  生徒名 | 希望コマ数合計")
        print("  -------|---------------")
        for sid, count in req_summary.items():
            print(f"  {s_map.get(sid, sid):<6} | {count} コマ")

        # 講師指定のチェック
        print("\n  [講師指定状況]")
        has_pref = False
        for _, row in df_reqs.iterrows():
            sid = row['student_id']
            cid = row['course_id']
            # 指定があるか確認
            prefs = []
            for i in range(1, 4):
                col = f'desired_teacher_{i}'
                if col in row and pd.notna(row[col]) and row[col] != '':
                    prefs.append(t_map.get(row[col], str(row[col])))

            if prefs:
                has_pref = True
                print(f"  - {s_map.get(sid)} ({c_map.get(cid)}): {', '.join(prefs)}")

        if not has_pref:
            print("  (講師指定は見つかりませんでした。全講師対象として計算します)")

    else:
        print("  ⚠️ リクエストデータがありません。")

    # --- 2. 生徒の空き状況チェック ---
    print(f"\n📌 【生徒の空き状況】(student_availability)")
    if not df_s_avail.empty:
        s_avail_count = df_s_avail.groupby('student_id').size()
        print("  生徒名 | 空いているスロット数")
        print("  -------|---------------------")
        for sid, count in s_avail_count.items():
            print(f"  {s_map.get(sid, sid):<6} | {count} 箇所")

        # 警告: 希望数に対して空きが少なすぎる生徒
        print("  ---(チェック)---")
        for sid in req_summary.index:
            req = req_summary.get(sid, 0)
            avail = s_avail_count.get(sid, 0)
            if avail < req:
                print(f"  ⚠️ 注意: {s_map.get(sid)}さんは 希望{req}コマ に対し、空きが {avail}箇所 しかありません！（物理的に配置不可）")
            elif avail == 0:
                print(f"  ⚠️ 注意: {s_map.get(sid)}さんの空き情報が登録されていません。")
    else:
        print("  ⚠️ 生徒の空きデータが空です！GASで出力しましたか？")

    # --- 3. 講師の空き状況チェック ---
    print(f"\n📌 【講師の空き状況】")
    if not df_t_avail.empty:
        t_avail_count = df_t_avail.groupby('teacher_id').size()
        print(f"  登録講師数: {len(t_avail_count)} 名")
        # 詳細表示したければコメントアウト解除
        # for tid, count in t_avail_count.items():
        #     print(f"  {t_map.get(tid):<6} | {count} 箇所")
    else:
        print("  ⚠️ 講師の空きデータがありません。")

    print("\n✅ データの確認が完了しました。")
    print("   問題なければ、次のセルで「最適化計算」を実行してください。")

except Exception as e:
    print(f"\n❌ エラーが発生しました: {e}")

--- 📥 データを読み込んでいます... ---
・lesson_slot: 12行 読み込みOK
・course: 4行 読み込みOK
・student_list: 6行 読み込みOK
・student_course: 9行 読み込みOK
・student_availability: 33行 読み込みOK
・teacher_list: 6行 読み込みOK
・teachable_courses: 14行 読み込みOK
・teacher_availability: 37行 読み込みOK

📊 データ診断レポート

📌 【授業リクエスト】(全 9 件)
  生徒名 | 希望コマ数合計
  -------|---------------
  s1     | 4 コマ
  s2     | 4 コマ
  s3     | 3 コマ
  s4     | 4 コマ
  s5     | 3 コマ
  s6     | 3 コマ

  [講師指定状況]
  - s1 (japanese): t1
  - s1 (math): t1
  - s2 (math): t3
  - s2 (science): t4
  - s3 (math): t5
  - s4 (japanese): t6
  - s4 (social): t1
  - s5 (math): t2
  - s6 (japanese): t3

📌 【生徒の空き状況】(student_availability)
  生徒名 | 空いているスロット数
  -------|---------------------
  s1     | 6 箇所
  s2     | 4 箇所
  s3     | 7 箇所
  s4     | 4 箇所
  s5     | 6 箇所
  s6     | 6 箇所
  ---(チェック)---

📌 【講師の空き状況】
  登録講師数: 6 名

✅ データの確認が完了しました。
   問題なければ、次のセルで「最適化計算」を実行してください。


In [9]:
# ==========================================
# 3. 最適化計算の実行とスプレッドシートへの保存 (講師指定・回数制限版)
# ==========================================

print("--- 🧠 最適化計算を開始します ---")

try:
    # 前処理（高速化用）
    slot_map = {row['id']: f"{row['date']} (S{row['time_range_id']})" for _, row in df_slots.iterrows()}

    # 指導可能辞書
    teachable_dict = collections.defaultdict(set)
    for _, row in df_teachable.iterrows():
        teachable_dict[row['teacher_id']].add(row['course_id'])

    # 空き状況セット
    student_avail_set = collections.defaultdict(set)
    for _, row in df_s_avail.iterrows():
        student_avail_set[row['student_id']].add(row['slot_id'])

    teacher_avail_set = collections.defaultdict(set)
    for _, row in df_t_avail.iterrows():
        teacher_avail_set[row['teacher_id']].add(row['slot_id'])

    # --- リクエスト情報の構築 (詳細化) ---
    requests = []
    # 講師ごとの制限情報を格納する辞書: limit_constraints[(sid, cid, tid)] = max_slots
    limit_constraints = {}

    for _, row in df_reqs.iterrows():
        sid = row['student_id']
        cid = row['course_id']
        sessions = row['sessions']

        # 希望講師リストを取得
        desired_teachers = []

        # 1〜3までループで取得
        for i in range(1, 4):
            t_col = f'desired_teacher_{i}'
            limit_col = f'max_slot_{i}'

            # 列が存在し、値が入っている場合のみ処理
            if t_col in row and pd.notna(row[t_col]) and row[t_col] != '':
                tid = row[t_col]
                desired_teachers.append(tid)

                # 上限数の取得
                if limit_col in row and pd.notna(row[limit_col]) and row[limit_col] != '':
                    limit = int(row[limit_col])
                    limit_constraints[(sid, cid, tid)] = limit
                else:
                    # 空欄の場合は制限なし（=必要なコマ数分まではOK）
                    limit_constraints[(sid, cid, tid)] = sessions

        # もし希望講師が一人も書かれていなければ、制約なしとして全講師を対象にする
        if not desired_teachers:
            desired_teachers = [t for t in all_teachers if cid in teachable_dict.get(t, set())]

        requests.append({
            'sid': sid,
            'cid': cid,
            'sessions': sessions,
            'allowed_teachers': desired_teachers
        })

    all_slots = df_slots['id'].tolist()
    all_teachers = df_teachers['id'].tolist()

    # ソルバー定義
    solver = pywraplp.Solver.CreateSolver('SCIP')
    solver.SetTimeLimit(30000) # 30秒制限

    # 変数作成
    x = {} # x[sid, cid, tid, slid]
    print("  変数を生成中...")

    for req in requests:
        sid, cid = req['sid'], req['cid']

        # ★指導可能かつ、希望リストに含まれている講師のみを対象にする
        candidate_teachers = [t for t in req['allowed_teachers'] if cid in teachable_dict.get(t, set())]

        for tid in candidate_teachers:
            # 共通の空きスロット
            common_slots = student_avail_set[sid].intersection(teacher_avail_set[tid])
            for slid in common_slots:
                x[(sid, cid, tid, slid)] = solver.IntVar(0, 1, f'x_{sid}_{cid}_{tid}_{slid}')

    print(f"  -> 生成された組み合わせ候補数: {len(x)}")

    # --- 制約条件 ---

    # 1. コマ数上限（合計）
    for req in requests:
        sid, cid, sessions = req['sid'], req['cid'], req['sessions']
        relevant_vars = [v for (s, c, t, sl), v in x.items() if s == sid and c == cid]
        if relevant_vars:
            solver.Add(solver.Sum(relevant_vars) <= sessions)

    # ★追加制約: 講師ごとのコマ数上限 (max_slot)
    for (sid, cid, tid), limit in limit_constraints.items():
        # この生徒・科目・講師に関連する変数を探す
        relevant_vars = [v for (s, c, t, sl), v in x.items() if s == sid and c == cid and t == tid]
        if relevant_vars:
            solver.Add(solver.Sum(relevant_vars) <= limit)

    # 2. 同時受講禁止（生徒）
    for sid in s_map.keys():
        for slid in all_slots:
            vars_s = [v for (s, c, t, sl), v in x.items() if s == sid and sl == slid]
            if vars_s: solver.Add(solver.Sum(vars_s) <= 1)

    # 3. 同時指導禁止（講師）
    for tid in t_map.keys():
        for slid in all_slots:
            vars_t = [v for (s, c, t, sl), v in x.items() if t == tid and sl == slid]
            if vars_t: solver.Add(solver.Sum(vars_t) <= 1)

    # 目的関数
    objective = solver.Objective()
    for v in x.values(): objective.SetCoefficient(v, 1)
    objective.SetMaximization()

    # 計算実行
    print("  計算中...")
    status = solver.Solve()

    if status in [pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE]:
        print("  ★ 計算完了。結果を集計します。")

        allocated = []
        counts = collections.defaultdict(int)

        # 検証用マップ
        student_busy = collections.defaultdict(set)
        teacher_busy = collections.defaultdict(set)

        for (sid, cid, tid, slid), v in x.items():
            if v.solution_value() > 0.5:
                allocated.append([
                    slid, sid, tid, cid,
                    slot_map.get(slid, str(slid)),
                    s_map.get(sid), t_map.get(tid), c_map.get(cid)
                ])
                counts[(sid, cid)] += 1
                student_busy[sid].add(slid)
                teacher_busy[tid].add(slid)

        # 未配置検証
        unallocated = []
        for req in requests:
            sid, cid, req_num = req['sid'], req['cid'], req['sessions']
            diff = req_num - counts[(sid, cid)]

            if diff > 0:
                reasons = []
                # 候補講師
                valid_ts = [t for t in req['allowed_teachers'] if cid in teachable_dict.get(t, set())]

                if not valid_ts:
                    reasons.append("指定講師が指導不可")

                # 空きはあるか検証
                has_slot_potential = False
                for tid in valid_ts:
                    base_common = student_avail_set[sid].intersection(teacher_avail_set[tid])

                    # 講師ごとの回数制限チェック
                    current_assign = len([row for row in allocated if row[1]==sid and row[3]==cid and row[2]==tid])
                    limit = limit_constraints.get((sid, cid, tid), 999)

                    if current_assign >= limit:
                         reasons.append(f"{t_map.get(tid)}:上限({limit})到達")
                         continue

                    for sl in base_common:
                        # 本来空いてるが、結果的に埋まったか？
                        if sl not in student_busy[sid] and sl not in teacher_busy[tid]:
                            reasons.append(f"空き枠あり(S{sl}-{t_map.get(tid)})")
                            has_slot_potential = True

                msg = "競合により枠確保できず"
                if not valid_ts: msg = "条件に合う講師なし"
                elif has_slot_potential: msg = "計算打ち切り等の可能性あり"

                # 理由を文字列化
                reason_str = "; ".join(reasons[:5])
                unallocated.append([sid, cid, diff, s_map.get(sid), c_map.get(cid), f"{msg} ({reason_str})"])

        # 出力
        df_out = pd.DataFrame(allocated, columns=['slot_id', 'student_id', 'teacher_id', 'course_id', '日時', '生徒名', '講師名', '科目名']).sort_values(['slot_id', 'student_id'])
        df_un = pd.DataFrame(unallocated, columns=['student_id', 'course_id', '不足数', '生徒名', '科目名', '理由'])

        print(f"\n✅ 配置成功: {len(df_out)} コマ")
        display(df_out[['日時', '生徒名', '講師名', '科目名']].head())

        if not df_un.empty:
            print(f"⚠️ 未配置: {len(df_un)} 件")
            display(df_un)

        # 書き込み
        def save_sheet(name, df):
            try:
                try: ws = wb.worksheet(name)
                except: ws = wb.add_worksheet(name, 1000, 20)
                ws.clear()
                if not df.empty: ws.update([df.columns.values.tolist()] + df.values.tolist())
                print(f"  シート '{name}' に保存しました。")
            except Exception as e: print(f"  書き込みエラー({name}): {e}")

        save_sheet('output_allocated_lessons', df_out)
        save_sheet('output_unallocated_lessons', df_un)

    else:
        print("❌ 計算できませんでした。条件を見直してください。")

except Exception as e:
    print(f"\n❌ 予期せぬエラー: {e}")

--- 🧠 最適化計算を開始します ---
  変数を生成中...
  -> 生成された組み合わせ候補数: 17
  計算中...
  ★ 計算完了。結果を集計します。

✅ 配置成功: 12 コマ


,日時,生徒名,講師名,科目名
0,2025-12-13 (S1),s1,t1,japanese
4,2025-12-13 (S2),s2,t4,science
1,2025-12-13 (S3),s1,t1,math
7,2025-12-13 (S4),s3,t5,math
10,2025-12-13 (S4),s6,t3,japanese


⚠️ 未配置: 4 件


,student_id,course_id,不足数,生徒名,科目名,理由
0,2,2,2,s2,math,条件に合う講師なし (指定講師が指導不可)
1,4,1,2,s4,japanese,条件に合う講師なし (指定講師が指導不可)
2,4,4,2,s4,social,条件に合う講師なし (指定講師が指導不可)
3,5,2,3,s5,math,条件に合う講師なし (指定講師が指導不可)


  シート 'output_allocated_lessons' に保存しました。
  シート 'output_unallocated_lessons' に保存しました。
